In [7]:

import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import sys
import pickle
sys.path.append(os.path.abspath('..'))  # Adds the parent directory to sys.path
from src import config
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import logging
# Set up logging
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def load_data():
    """Loads data from the SQLite database."""
    conn = sqlite3.connect(config.DATABASE_PATH)
    query = f"SELECT latitude, longitude, price FROM {config.PROCESSED_TABLE}"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


def train_model(grid_search=False):
    """Trains a Random Forest model with GridSearchCV and saves evaluation metrics to CSV."""
    df = load_data().head(1000)

    # Save original indices before vectorization
    df_indices = df.index

    # Feature extraction
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['cleaned_text'])
    y = df['Y house price of unit area']

    with open(f"{config.MODELS_PATH}vectorizer.pkl", 'wb') as f:
        pickle.dump(vectorizer, f)


    # Train-test split (preserve indices)
    X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
        X, y, df_indices, test_size=0.2, random_state=42
    )

    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    #y_pred = rf.predict(X_test)
        
    # Inizializza i modelli
    lr_model = LinearRegression()

# Addestra i modelli

    lr_model.fit(X_train, y_train)

# Predizioni sui dati di test
    rf_pred = rf.predict(X_test)
    lr_pred = lr_model.predict(X_test)

    # Funzione per calcolare le metriche di valutazione
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"📊 {model_name} Metrics:")
    print(f"🔹 Mean Squared Error (MSE): {mse:.4f}")
    print(f"🔹 Mean Absolute Error (MAE): {mae:.4f}")
    print(f"🔹 R² Score: {r2:.4f}")
    print("-" * 40)

# Valutazione dei modelli
    evaluate_model(y_test, rf_pred, "Random Forest")
    evaluate_model(y_test, lr_pred, "Linear Regression")


In [8]:
"""
# Valutazione dei modelli
evaluate_model(y_test, rf_pred, "Random Forest")
evaluate_model(y_test, lr_pred, "Linear Regression")
    # Nayve Bayes
 #   NB=linear_model.GaussianNB()
  #  NB.fit(X,y)()
   # y_pred = rf.predict(X)

    logging.info('saving models')
#    with open(os.path.join(config.MODELS_PATH, "random_forest.pkl"), 'wb') as file:
#        pickle.dump(rf, file)   
    with open("../models/random_forest.pkl", 'wb') as file:
        pickle.dump(rf, file)
    with open("../models/logistic_regression.pkl", 'wb') as file:
        pickle.dump(rf, file)
#    with open("../models/Nayve_bayes.pkl", 'wb') as file:
 #       pickle.dump(rf, file)
    # Create a DataFrame for the test set with predictions
    test_df = df.loc[test_idx].copy()  # Copy test set rows
    test_df['prediction'] = y_pred  # Add predictions

    
 
    

    # Compute metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
        'recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
        'f1_score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
    }

    # Connect to the database
    conn = sqlite3.connect(config.DATABASE_PATH)

    # saving predictions
    test_df.to_sql(config.PREDICTIONS_TABLE, conn, if_exists='replace', index=False)
    
    # saving grid search results
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_sql(config.EVALUATION_TABLE, conn,
                      if_exists='replace', index=False)
    # Commit and close the connection
    conn.commit()
    conn.close()

"""

'\n# Valutazione dei modelli\nevaluate_model(y_test, rf_pred, "Random Forest")\nevaluate_model(y_test, lr_pred, "Linear Regression")\n    # Nayve Bayes\n #   NB=linear_model.GaussianNB()\n  #  NB.fit(X,y)()\n   # y_pred = rf.predict(X)\n\n    logging.info(\'saving models\')\n#    with open(os.path.join(config.MODELS_PATH, "random_forest.pkl"), \'wb\') as file:\n#        pickle.dump(rf, file)   \n    with open("../models/random_forest.pkl", \'wb\') as file:\n        pickle.dump(rf, file)\n    with open("../models/logistic_regression.pkl", \'wb\') as file:\n        pickle.dump(rf, file)\n#    with open("../models/Nayve_bayes.pkl", \'wb\') as file:\n #       pickle.dump(rf, file)\n    # Create a DataFrame for the test set with predictions\n    test_df = df.loc[test_idx].copy()  # Copy test set rows\n    test_df[\'prediction\'] = y_pred  # Add predictions\n\n\n\n\n\n    # Compute metrics\n    metrics = {\n        \'accuracy\': accuracy_score(y_test, y_pred),\n        \'precision\': precisi